### Value/spatial pattern/time series of variable x in future scenario in model y in scenario 1, 2, 3 

Author(s): Sofiane Bendoukha / Karsten Peters 

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3397361.svg)](https://doi.org/10.5281/zenodo.3397361)

#### Question 1:

What is the global near surface air temperature simulated by “a particular model” under the RCP2.6 and RCP8.5 scenarios at the end of the century - and what is the difference between the two?


Connect to remote ECAS instance

In [ ]:
from PyOphidia import cube
cube.Cube.setclient(read_env=True)

 1. Load RCP2.6 and RCP8.5 into OPhidia

In [ ]:
rcp26 = cube.Cube(src_path='/mnt/lustre01/work/kd0956/CMIP5/data/cmip5/output1/MOHC/HadGEM2-ES/rcp26/day/atmos/day/r1i1p1/v20120405/tas/tas_day_HadGEM2-ES_rcp26_r1i1p1_20951201-20991130.nc',
    measure='tas',
    import_metadata='yes',
    imp_dim='time',
    vocabulary='CF',hierarchy='oph_base|oph_base|oph_time', 
    ncores=4
    )

In [ ]:
#rcp26.info()

In [ ]:
rcp85 = cube.Cube(src_path='/mnt/lustre01/work/kd0956/CMIP5/data/cmip5/output1/MOHC/HadGEM2-ES/rcp85/day/atmos/day/r1i1p1/v20111128/tas/tas_day_HadGEM2-ES_rcp85_r1i1p1_20951201-20991130.nc',
    measure='tas',
    import_metadata='yes',
    imp_dim='time',
    vocabulary='CF',hierarchy='oph_base|oph_base|oph_time',
    ncores=4,
    description='Air temperatures'
    )

In [ ]:
# rcp85.info()


2- Compute time mean (rcp85, rcp26)

In [ ]:
tas_rcp85 = rcp85.reduce2(
    dim='time',
    concept_level='y',
    operation='avg',
    description='Mean rcp85'
) 

In [ ]:
tas_rcp26 = rcp26.reduce2(
    dim='time',
    concept_level='y',
    operation='avg',
    description='Mean rcp26'
)

In [ ]:
#tas_rcp85.info()

3- Compute difference tas_rcp85 - tas_rcp26

In [ ]:
dtas = tas_rcp85.intercube(
    cube2=tas_rcp26.pid,
    operation='sub',
    ncores=4,
    description='Differences RCP85 and RCP26')

In [ ]:
#dtas.info()

Extract first year

In [ ]:
firstYearDtas = dtas.subset2(subset_filter='2099', subset_dims='time')
data = firstYearDtas.export_array()

Plot dtas

Plot the global map (first year)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from mpl_toolkits.basemap import Basemap, cm

lat = data['dimension'][0]['values'][:]
lon = data['dimension'][1]['values'][:]
var = data['measure'][0]['values'][:]
var = np.reshape(var, (len(lat), len(lon)))

fig = plt.figure(figsize=(15, 15), dpi=100)
ax  = fig.add_axes([0.1,0.1,0.8,0.8])

map = Basemap(projection='cyl',llcrnrlat= -90,urcrnrlat= 90, llcrnrlon=0,urcrnrlon=360, resolution='c')

map.drawcoastlines()
map.drawparallels(np.arange( -90, 90,30),labels=[1,0,0,0])
map.drawmeridians(np.arange(-180,180,30),labels=[0,0,0,1])

x, y = map(*np.meshgrid(lon,lat))

clevs = np.arange(-10,25,1)

cnplot = map.contourf(x,y,var,clevs,cmap=plt.cm.jet)
cbar = map.colorbar(cnplot,location='right')

plt.title('DTAS (deg K)')
plt.show()

In [ ]:
#firstYearDtas.info()

### Scientific description
In CMIP5, the models were run under certain boundary conditions, i.e. the forcing scenarios. In our case here, RCP2.6 involves much less greenhouse gas forcing than RCP8.5, so we expect that the temperature in the RCP8.5 simulation is significantly higher than the one simulated by the RCP2.6 simulation. This is also what we show in the plots. In particular, the difference plot (RCP8.5 - RCP2.6) indicates that the RCP8.5 simulation simulates a higher near surface temperature practically everywhere. We can also see, that the difference becomes larger towards the poles - the effect commonly known as “arctic amplification”.


=============================
### Question 2

What is the temporal evolution of global mean near surface air temperature from 2005 - 2100 under forcing scenarios RCP2.6 and RCP8.5?


#### 1-a Import list of rcp26 files (Terminal)

In [ ]:
cube.Cube.createcontainer(container='rcp26',base_time='2005-12-01', dim='lat|lon|time',dim_type='double|double|double',hierarchy='oph_base|oph_base|oph_time')
cube.Cube.createcontainer(container='rcp26_mean',dim='lat|lon|time', base_time='2005-12-01', dim_type='double|double|double',hierarchy='oph_base|oph_base|oph_time')

In [ ]:
cube.Cube.client.submit("oph_importnc src_path=[path=/mnt/lustre01/work/kd0956/CMIP5/data/cmip5/output1/MOHC/HadGEM2-ES/rcp26/mon/atmos/Amon/r1i1p1/v20111206/tas/;file=tas_Amon_HadGEM2-ES_rcp26*.nc];imp_dim=time;imp_concept_level=M;vocabulary=CF;hierarchy=oph_base|oph_base|oph_time;ncores=4;measure=tas;base_time=2005-12-01;container=rcp26;") 

In [ ]:
#cube.Cube.list(level=2)

#### 1-b Calculate mean for each nc file 

In [ ]:
cube.Cube.client.submit("oph_reduce2 cube=[container=rcp26;level=0;];container=rcp26_mean;operation=avg;dim=time;concept_level=y;ncores=4;")

#### 1-c Merge all cubes in one

In [ ]:
# Calculate cube list from a container 
import json 
def listCubes(container):
    list_cubes = []
    # return json-based oph_list with level=2 of the container rcp26_mean
    cube.Cube.list(container_filter=container, level=2, display=False)
    json_response = json.loads(cube.Cube.client.last_response)
    list_cubes_str = json_response['response'][0]['objcontent'][0]['rowvalues']
    list_cubes.append(list_cubes_str[0][2])
    
    for i in range(len(list_cubes_str)):
        list_cubes.append(list_cubes_str[i][2])
    seperator = '|'
    return seperator.join(list_cubes)

In [ ]:
list_cubes = listCubes('rcp26_mean')
merged_cube = cube.Cube.mergecubes(cubes=list_cubes,mode='a', hold_values='yes')

In [ ]:
#merged_cube.info()

In [ ]:
#merged_cube.exportnc2(output_name='merged_cube', output_path='/data-training')

In [ ]:
firstyear = merged_cube.subset2(subset_filter=1, subset_dims='time')

In [ ]:
data = firstyear.export_array(show_time='yes')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import numpy as np

lat = data['dimension'][0]['values'][:]
lon = data['dimension'][1]['values'][:]
var = data['measure'][0]['values'][:]
var = np.reshape(var, (len(lat), len(lon)))

fig = plt.figure(figsize=(15, 15), dpi=100)
ax  = fig.add_axes([0.1,0.1,0.8,0.8])

map = Basemap(projection='cyl',llcrnrlat= -90,urcrnrlat= 90, llcrnrlon=0,urcrnrlon=360, resolution='c')

map.drawcoastlines()
map.drawparallels(np.arange( -90, 90,30),labels=[1,0,0,0])
map.drawmeridians(np.arange(-180,180,30),labels=[0,0,0,1])

x, y = map(*np.meshgrid(lon,lat))

clevs = np.arange(0,371,10)

cnplot = map.contourf(x,y,var,clevs,cmap=plt.cm.jet)
cbar = map.colorbar(cnplot,location='right')

plt.title('Merged cube rcp 2.6 (first year)')
plt.show()

#### 2-a Import list of rcp85 files (Terminal)

In [ ]:
cube.Cube.createcontainer(container='rcp85',dim='lat|lon|time',dim_type='double|double|double',hierarchy='oph_base|oph_base|oph_time')
cube.Cube.createcontainer(container='rcp85_mean',dim='lat|lon|time',dim_type='double|double|double',hierarchy='oph_base|oph_base|oph_time')

In [ ]:
cube.Cube.client.submit("oph_importnc src_path=[path=/mnt/lustre01/work/kd0956/CMIP5/data/cmip5/output1/MOHC/HadGEM2-ES/rcp85/mon/atmos/Amon/r1i1p1/v20130430/tas/;file=tas_Amon_HadGEM2-ES_rcp85*.nc];imp_dim=time;imp_concept_level=M;vocabulary=CF;hierarchy=oph_base|oph_base|oph_time;ncores=4;measure=tas;container=rcp85;") 

In [ ]:
#cube.Cube.list(level=2)

#### 2-b Calculate mean for each nc file 

In [ ]:
cube.Cube.client.submit("oph_reduce2 cube=[container=rcp85;level=0;];container=rcp85_mean;operation=avg;dim=time;concept_level=y;ncores=4;")

#### 2-c Merge all cubes in one

In [ ]:
# Calculate cube list from a container 
import json 
def listCubes(container):
    list_cubes = []
    # return json-based oph_list with level=2 of the container rcp26_mean
    cube.Cube.list(container_filter=container, level=2, display=False)
    json_response = json.loads(cube.Cube.client.last_response)
    list_cubes_str = json_response['response'][0]['objcontent'][0]['rowvalues']
    list_cubes.append(list_cubes_str[0][2])
    
    for i in range(len(list_cubes_str)):
        list_cubes.append(list_cubes_str[i][2])
    seperator = '|'
    return seperator.join(list_cubes)

In [ ]:
list_cubes = listCubes('rcp85_mean')
merged_cube_rcp85 = cube.Cube.mergecubes(cubes=list_cubes,mode='a', hold_values='yes')

In [ ]:
merged_cube_rcp85.info()

In [ ]:
#merged_cube.exportnc2(output_name='merged_cube', output_path='/data-training')

In [ ]:
firstyear = merged_cube.subset2(subset_filter=1, subset_dims='time')

In [ ]:
data = firstyear.export_array(show_time='yes')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import numpy as np

lat = data['dimension'][0]['values'][:]
lon = data['dimension'][1]['values'][:]
var = data['measure'][0]['values'][:]
var = np.reshape(var, (len(lat), len(lon)))

fig = plt.figure(figsize=(15, 15), dpi=100)
ax  = fig.add_axes([0.1,0.1,0.8,0.8])

map = Basemap(projection='cyl',llcrnrlat= -90,urcrnrlat= 90, llcrnrlon=0,urcrnrlon=360, resolution='c')

map.drawcoastlines()
map.drawparallels(np.arange( -90, 90,30),labels=[1,0,0,0])
map.drawmeridians(np.arange(-180,180,30),labels=[0,0,0,1])

x, y = map(*np.meshgrid(lon,lat))

clevs = np.arange(0,371,10)

cnplot = map.contourf(x,y,var,clevs,cmap=plt.cm.jet)
cbar = map.colorbar(cnplot,location='right')

plt.title('Merged cube rcp 8.5 (first year)')
plt.show()

#### Clean your workspace 

In [ ]:
cube.Cube.client.submit("oph_delete cube=[container=rcp85]")
cube.Cube.deletecontainer(container='rcp85')

#### Scientific description
Similar to the first example of this use case, we “know” that the temperature simulated by the RCP8.5 scenario should and will always be higher throughout the climate projection. This is visible in the time series plots we produce in this exercise. The difference between the two scenario simulations, i.e. the difference between the two curves, will increase with time, i.e. the two simulations will be closer together in 2010 compared to 2100 (at least this is what we should be seeing). Further, the curves will not be straight lines but show “wiggles”. This is a sign of internal variability, i.e. some years are colder than others. This is also expected. In short, while the first part of use case 1 gives us only a snapshot of temperature changes on a global map, the second exercise gives us an impression of the transient response of the model to different forcing scenarios throughout time.
